In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input
from keras.layers import Embedding, LSTM
from keras.models import Sequential

import numpy as np
import csv

Using TensorFlow backend.


In [2]:
GLOVE_PATH = ''
TRAIN_DATA_PATH = ''

SENTENCE_LENGTH = 100
EMBEDDING_DIM = 50
TEST_SPLIT = 0.2

VOCAB_SIZE = 1000

In [3]:
def loadGlove(path):
    embeddings_index = {}
    with open(path, "r", encoding="utf8") as glovef:
        for line in glovef:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

In [4]:
def loadDataSet(path):
    label = []
    data = []

    with open(path,'r') as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t')
        for row in tsvin:
            label.append(int(row[0]))
            data.append(row[1])
    return data, label

In [5]:
def split(ratio, data, labels):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    
    data = data[indices]
    labels = labels[indices]
    
    nb_test = int(ratio * data.shape[0])

    x_train = data[:-nb_test]
    y_train = labels[:-nb_test]

    x_test = data[-nb_test:]
    y_test = labels[-nb_test:]
    
    return x_train, y_train, x_test, y_test

In [6]:
embeddings_table = loadGlove(GLOVE_PATH)
data_lines, data_labels = loadDataSet(TRAIN_DATA_PATH)

Found 400000 word vectors.


In [7]:
tokenizer = Tokenizer(nb_words=VOCAB_SIZE)
tokenizer.fit_on_texts(data_lines)
data_sequences = tokenizer.texts_to_sequences(data_lines)
data_index = tokenizer.word_index
vocab_dim = min(VOCAB_SIZE, len(data_index))


In [8]:
data_padded = pad_sequences(data_sequences, maxlen=SENTENCE_LENGTH)
data_labels = np.asarray(data_labels)

In [9]:
x_train, y_train, x_test, y_test = split(TEST_SPLIT, data_padded, data_labels)

In [10]:
embedding_matrix = np.zeros((vocab_dim, EMBEDDING_DIM))

for word, i in data_index.items():
    if i < vocab_dim: 
        embedding_vector = embeddings_table.get(word)

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# save the initial word embeddings
#np.save('init_embedding.npy', embedding_matrix)
init_weights = np.copy(embedding_matrix)    
embedding_layer = Embedding(vocab_dim, EMBEDDING_DIM, weights=[embedding_matrix], input_length=SENTENCE_LENGTH, trainable=True)

In [11]:
print('Training model.')

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2, return_sequences=True)) 
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test),nb_epoch=20, batch_size=20)


Training model.
Train on 1260 samples, validate on 314 samples
Epoch 1/20
1260/1260 [==============================] - 19s - loss: 0.7194 - acc: 0.5262 - val_loss: 0.7105 - val_acc: 0.4841
Epoch 2/20
1260/1260 [==============================] - 19s - loss: 0.6873 - acc: 0.5405 - val_loss: 0.6847 - val_acc: 0.5764
Epoch 3/20
1260/1260 [==============================] - 20s - loss: 0.6758 - acc: 0.5675 - val_loss: 0.6728 - val_acc: 0.5955
Epoch 4/20
1260/1260 [==============================] - 21s - loss: 0.6541 - acc: 0.6143 - val_loss: 0.6544 - val_acc: 0.6019
Epoch 5/20
1260/1260 [==============================] - 21s - loss: 0.6247 - acc: 0.6627 - val_loss: 0.6240 - val_acc: 0.6401
Epoch 6/20
1260/1260 [==============================] - 22s - loss: 0.5802 - acc: 0.6976 - val_loss: 0.6793 - val_acc: 0.6083
Epoch 7/20
1260/1260 [==============================] - 21s - loss: 0.5636 - acc: 0.7095 - val_loss: 0.5955 - val_acc: 0.6752
Epoch 8/20
1260/1260 [==============================] -

In [16]:
# train linear classifier between the inti and final embedding
trained_embeeding = np.copy(embedding_layer.get_weights())

expansion_model = Sequential()
expansion_model.add(Dense(EMBEDDING_DIM,init='uniform', activation='linear', input_dim=EMBEDDING_DIM))

expansion_model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
expansion_model.fit(init_weights, trained_embeeding.reshape(VOCAB_SIZE,EMBEDDING_DIM), nb_epoch=100, batch_size=20)

Epoch 1/100
1000/1000 [==============================] - 0s - loss: 0.3723 - acc: 0.5560     
Epoch 2/100
1000/1000 [==============================] - 0s - loss: 0.2538 - acc: 0.8170     
Epoch 3/100
1000/1000 [==============================] - 0s - loss: 0.1843 - acc: 0.8360     
Epoch 4/100
1000/1000 [==============================] - 0s - loss: 0.1380 - acc: 0.8440     
Epoch 5/100
1000/1000 [==============================] - 0s - loss: 0.1059 - acc: 0.8530     
Epoch 6/100
1000/1000 [==============================] - 0s - loss: 0.0839 - acc: 0.8590     
Epoch 7/100
1000/1000 [==============================] - 0s - loss: 0.0676 - acc: 0.8650     
Epoch 8/100
1000/1000 [==============================] - 0s - loss: 0.0546 - acc: 0.8740     
Epoch 9/100
1000/1000 [==============================] - 0s - loss: 0.0439 - acc: 0.8810     
Epoch 10/100
1000/1000 [==============================] - 0s - loss: 0.0350 - acc: 0.8910     
Epoch 11/100
1000/1000 [==============================] - 0

In [17]:
expansion_model.save('EmbeddingExpansion.pkl')